# Flood Estimate Code

This code produces flood heights for given storm parameters. The code calculates surface volume functions that can relate the volume of water to the flood height at a given division. Using Mannning's equation, it takes topographic data and storm parameter data to find the velocity and subsequently the volume of the water. This volumes are then redistributed by propagating to the nearby divisions, and the final flood heights for each division are determined.


Inputs: surface data (topography, roughness, slope, surface volume, and surface volume grouped), flood data for cold and warm storms (peak, surge, time), and wall data for cold and warm storms.

Outputs: Flood heights for cold and warm storms with and without flood wall protection in csv format.

## Imports and Functions

In [1]:
import numpy as np
import pandas as pd
import time
import csv
import glob                                                                                                
import datetime

In [2]:
# essentials
ftm = 0.3048 # feet to meter
nt  = 10     # split number of time history of storm surge

In [3]:
## Functions

# curve fitting
from scipy.optimize import curve_fit
def func_fit(surfaceV,a,b):
    return a*np.sqrt(surfaceV)+b*surfaceV


def SurfaceVolFunc(surfaceV,H):
    """
    This function takes the surface volume values calculated using the Surface Volume Processor
    and interpolates the values into a surface volume function.
    """
    
    a1 = 0;  a2 = 2;  b1 = 1;  b2 = 3;     c1 = 2;  c2 = 4;  d1 = 3;  d2 = 5
    e1 = 4;  e2 = 6;  f1 = 5;  f2 = 7;     g1 = 6;  g2 = 8;  h1 = 7;  h2 = 9
    i1 = 8;  i2 = 10; j1 = 9;  j2 = 11;    k1 = 10; k2 = 12; l1 = 11; l2 = 13
    m1 = 12; m2 = 14; n1 = 13; n2 = 15;    o1 = 14; o2 = 16; p1 = 15; p2 = 17
    q1 = 16; q2 = 18; r1 = 17; r2 = 19;    s1 = 18; s2 = 20; t1 = 19; t2 = 21
    
    popt1, pcov1 = curve_fit(func_fit, surfaceV[a1:a2], H[a1:a2])
    popt2, pcov2 = curve_fit(func_fit, surfaceV[b1:b2], H[b1:b2])
    popt3, pcov3 = curve_fit(func_fit, surfaceV[c1:c2], H[c1:c2])
    popt4, pcov4 = curve_fit(func_fit, surfaceV[d1:d2], H[d1:d2])
    popt5, pcov5 = curve_fit(func_fit, surfaceV[e1:e2], H[e1:e2])
    popt6, pcov6 = curve_fit(func_fit, surfaceV[f1:f2], H[f1:f2])
    popt7, pcov7 = curve_fit(func_fit, surfaceV[g1:g2], H[g1:g2])
    popt8, pcov8 = curve_fit(func_fit, surfaceV[h1:h2], H[h1:h2])
    popt9, pcov9 = curve_fit(func_fit, surfaceV[i1:i2], H[i1:i2])
    popt10, pcov10 = curve_fit(func_fit, surfaceV[j1:j2], H[j1:j2])
    popt11, pcov11 = curve_fit(func_fit, surfaceV[k1:k2], H[k1:k2])
    popt12, pcov12 = curve_fit(func_fit, surfaceV[l1:l2], H[l1:l2])
    popt13, pcov13 = curve_fit(func_fit, surfaceV[m1:m2], H[m1:m2])
    popt14, pcov14 = curve_fit(func_fit, surfaceV[n1:n2], H[n1:n2])
    popt15, pcov15 = curve_fit(func_fit, surfaceV[o1:o2], H[o1:o2])
    popt16, pcov16 = curve_fit(func_fit, surfaceV[p1:p2], H[p1:p2])
    popt17, pcov17 = curve_fit(func_fit, surfaceV[q1:q2], H[q1:q2])
    popt18, pcov18 = curve_fit(func_fit, surfaceV[r1:r2], H[r1:r2])
    popt19, pcov19 = curve_fit(func_fit, surfaceV[s1:s2], H[s1:s2])
    popt20, pcov20 = curve_fit(func_fit, surfaceV[t1:t2], H[t1:t2])
            
    return popt1,popt2,popt3,popt4,popt5,popt6,popt7,popt8,popt9,popt10,popt11,popt12,popt13,popt14,popt15,popt16,popt17,popt18,popt19,popt20

def FloodHeight(surfaceV,slope,roughness,SVf1,SVf2,SVf3,SVf4,SVf5,SVf6,SVf7,SVf8,SVf9,SVf10,SVf11,SVf12,SVf13,SVf14,SVf15,SVf16,SVf17,SVf18,SVf19,SVf20,time1_w,time2_w,cpi1_w,cpi2_w,nt,elev,fid,l,ssh,i):
    """
    This function calculates flood heights for a given div without considering flood water redistribution
    """
    
    
    sec1_w = time1_w * 60**2; sec2_w = time2_w * 60**2  #time in seconds
    h1_w = cpi1_w;            h2_w = cpi2_w
    V_new = 0; v_new = np.zeros(nt*2)
    
    a1 = 0;  a2 = 2;  b1 = 1;  b2 = 3;     c1 = 2;  c2 = 4;  d1 = 3;  d2 = 5
    e1 = 4;  e2 = 6;  f1 = 5;  f2 = 7;     g1 = 6;  g2 = 8;  h1 = 7;  h2 = 9
    i1 = 8;  i2 = 10; j1 = 9;  j2 = 11;    k1 = 10; k2 = 12; l1 = 11; l2 = 13
    m1 = 12; m2 = 14; n1 = 13; n2 = 15;    o1 = 14; o2 = 16; p1 = 15; p2 = 17
    q1 = 16; q2 = 18; r1 = 17; r2 = 19;    s1 = 18; s2 = 20; t1 = 19; t2 = 21

    for k in fid:
        ds1 = np.tile(sec1_w[1] - sec1_w[0],(nt,1)).transpose() #delta between times
        ds2 = np.tile(sec2_w[1] - sec2_w[0],(nt,1)).transpose()
        ds  = np.concatenate((ds1,ds2),axis=1)

        h1_new = h1_w - elev[k] #difference betwwen surge heights anda elevations
        h2_new = h2_w - elev[k]
        h_new  = np.concatenate((h1_new,h2_new),axis=0)
        v_new[h_new > 0] = ((l * h_new[h_new > 0]) / (l + (2*h_new[h_new > 0])))**(2./3.)*slope**.5 / roughness #Manning's Equation
        v_new[h_new <= 0] = 0

        V_new = V_new + np.sum(h_new*l*v_new*ds) #volume to plug into surface volume functions to determin flood heights
    
    #determine flood heights based on surface volume function
    fld_h = np.zeros(np.shape(V_new))
    fld_h [V_new == 0]= 0
    fld_h [(V_new <= surfaceV[a2])  &  (V_new > 0)] = func_fit(V_new[(V_new <= surfaceV[a2])  &  (V_new > 0)],*SVf1)
    fld_h [(V_new <= surfaceV[b2])  &  (V_new > surfaceV[a2])] = func_fit(V_new[(V_new <= surfaceV[b2])  &  (V_new > surfaceV[a2])],*SVf2)
    fld_h [(V_new <= surfaceV[c2])  &  (V_new > surfaceV[b2])] = func_fit(V_new[(V_new <= surfaceV[c2])  &  (V_new > surfaceV[b2])],*SVf3)
    fld_h [(V_new <= surfaceV[d2])  &  (V_new > surfaceV[c2])] = func_fit(V_new[(V_new <= surfaceV[d2])  &  (V_new > surfaceV[c2])],*SVf4)
    fld_h [(V_new <= surfaceV[e2])  &  (V_new > surfaceV[d2])] = func_fit(V_new[(V_new <= surfaceV[e2])  &  (V_new > surfaceV[d2])],*SVf5)
    fld_h [(V_new <= surfaceV[f2])  &  (V_new > surfaceV[e2])] = func_fit(V_new[(V_new <= surfaceV[f2])  &  (V_new > surfaceV[e2])],*SVf6)
    fld_h [(V_new <= surfaceV[g2])  &  (V_new > surfaceV[f2])] = func_fit(V_new[(V_new <= surfaceV[g2])  &  (V_new > surfaceV[f2])],*SVf7)
    fld_h [(V_new <= surfaceV[h2])  &  (V_new > surfaceV[f2])] = func_fit(V_new[(V_new <= surfaceV[h2])  &  (V_new > surfaceV[f2])],*SVf8)
    fld_h [(V_new <= surfaceV[i2])  &  (V_new > surfaceV[h2])] = func_fit(V_new[(V_new <= surfaceV[i2])  &  (V_new > surfaceV[h2])],*SVf9)
    fld_h [(V_new <= surfaceV[j2])  &  (V_new > surfaceV[i2])] = func_fit(V_new[(V_new <= surfaceV[j2])  &  (V_new > surfaceV[i2])],*SVf10)
    fld_h [(V_new <= surfaceV[k2])  &  (V_new > surfaceV[j2])] = func_fit(V_new[(V_new <= surfaceV[k2])  &  (V_new > surfaceV[j2])],*SVf11)
    fld_h [(V_new <= surfaceV[l2])  &  (V_new > surfaceV[k2])] = func_fit(V_new[(V_new <= surfaceV[l2])  &  (V_new > surfaceV[k2])],*SVf12)
    fld_h [(V_new <= surfaceV[m2])  &  (V_new > surfaceV[l2])] = func_fit(V_new[(V_new <= surfaceV[m2])  &  (V_new > surfaceV[l2])],*SVf13)
    fld_h [(V_new <= surfaceV[n2])  &  (V_new > surfaceV[m2])] = func_fit(V_new[(V_new <= surfaceV[n2])  &  (V_new > surfaceV[m2])],*SVf14)
    fld_h [(V_new <= surfaceV[o2])  &  (V_new > surfaceV[n2])] = func_fit(V_new[(V_new <= surfaceV[o2])  &  (V_new > surfaceV[n2])],*SVf15)
    fld_h [(V_new <= surfaceV[p2])  &  (V_new > surfaceV[o2])] = func_fit(V_new[(V_new <= surfaceV[p2])  &  (V_new > surfaceV[o2])],*SVf16)
    fld_h [(V_new <= surfaceV[q2])  &  (V_new > surfaceV[p2])] = func_fit(V_new[(V_new <= surfaceV[q2])  &  (V_new > surfaceV[p2])],*SVf17)
    fld_h [(V_new <= surfaceV[r2])  &  (V_new > surfaceV[q2])] = func_fit(V_new[(V_new <= surfaceV[r2])  &  (V_new > surfaceV[q2])],*SVf18)
    fld_h [(V_new <= surfaceV[s2])  &  (V_new > surfaceV[r2])] = func_fit(V_new[(V_new <= surfaceV[s2])  &  (V_new > surfaceV[r2])],*SVf19)
    fld_h [V_new >= surfaceV[s2]] = func_fit(V_new[V_new > surfaceV[s2]],*SVf20)
    fld_h [fld_h < 0]= 0
    
    return fld_h,V_new

def FloodHeightWall(surfaceV,ParWall,slope,roughness,SVf1,SVf2,SVf3,SVf4,SVf5,SVf6,SVf7,SVf8,SVf9,SVf10,SVf11,SVf12,SVf13,SVf14,SVf15,SVf16,SVf17,SVf18,SVf19,SVf20,time1_w,time2_w,cpi1_w,cpi2_w,nt,elev,fid,l,ssh,i):
    """
    This function calculates flood heights with an altered Manning's Equation to account for the height of the wall.
    """
    
    sec1_w = time1_w * 60**2; sec2_w = time2_w * 60**2
    h1_w = cpi1_w;            h2_w = cpi2_w
    V_new = 0; v_new = np.zeros(nt*2)
    
    a1 = 0;  a2 = 2;  b1 = 1;  b2 = 3;     c1 = 2;  c2 = 4;  d1 = 3;  d2 = 5
    e1 = 4;  e2 = 6;  f1 = 5;  f2 = 7;     g1 = 6;  g2 = 8;  h1 = 7;  h2 = 9
    i1 = 8;  i2 = 10; j1 = 9;  j2 = 11;    k1 = 10; k2 = 12; l1 = 11; l2 = 13
    m1 = 12; m2 = 14; n1 = 13; n2 = 15;    o1 = 14; o2 = 16; p1 = 15; p2 = 17
    q1 = 16; q2 = 18; r1 = 17; r2 = 19;    s1 = 18; s2 = 20; t1 = 19; t2 = 21

    for k in fid:
        if k in ParWall:
            ds1 = np.tile(sec1_w[1] - sec1_w[0],(nt,1)).transpose()
            ds2 = np.tile(sec2_w[1] - sec2_w[0],(nt,1)).transpose()
            ds  = np.concatenate((ds1,ds2),axis=1)

            h1_new = h1_w - elev[k]
            h2_new = h2_w - elev[k]
            h_new  = np.concatenate((h1_new,h2_new),axis=0)
            cwr = 0.611 + 0.075*h_new[h_new > 0]/elev[k]            
            v_new[h_new > 0] = ((l * h_new[h_new > 0]) / (l + (2*h_new[h_new > 0])))**(2./3.)*slope**.5 / roughness * cwr
            v_new[h_new <= 0] = 0
            V_new = V_new + np.sum(h_new*l*v_new*ds)
            
        else:
            ds1 = np.tile(sec1_w[1] - sec1_w[0],(nt,1)).transpose()
            ds2 = np.tile(sec2_w[1] - sec2_w[0],(nt,1)).transpose()
            ds  = np.concatenate((ds1,ds2),axis=1)

            h1_new = h1_w - elev[k]
            h2_new = h2_w - elev[k]
            h_new  = np.concatenate((h1_new,h2_new),axis=0)
            v_new[h_new > 0] = ((l * h_new[h_new > 0]) / (l + (2*h_new[h_new > 0])))**(2./3.)*slope**.5 / roughness
            v_new[h_new <= 0] = 0
            V_new = V_new + np.sum(h_new*l*v_new*ds)
        
    fld_h = np.zeros(np.shape(V_new))
    fld_h [V_new == 0]= 0
    fld_h [(V_new <= surfaceV[a2])  &  (V_new > 0)] = func_fit(V_new[(V_new <= surfaceV[a2])  &  (V_new > 0)],*SVf1)
    fld_h [(V_new <= surfaceV[b2])  &  (V_new > surfaceV[a2])] = func_fit(V_new[(V_new <= surfaceV[b2])  &  (V_new > surfaceV[a2])],*SVf2)
    fld_h [(V_new <= surfaceV[c2])  &  (V_new > surfaceV[b2])] = func_fit(V_new[(V_new <= surfaceV[c2])  &  (V_new > surfaceV[b2])],*SVf3)
    fld_h [(V_new <= surfaceV[d2])  &  (V_new > surfaceV[c2])] = func_fit(V_new[(V_new <= surfaceV[d2])  &  (V_new > surfaceV[c2])],*SVf4)
    fld_h [(V_new <= surfaceV[e2])  &  (V_new > surfaceV[d2])] = func_fit(V_new[(V_new <= surfaceV[e2])  &  (V_new > surfaceV[d2])],*SVf5)
    fld_h [(V_new <= surfaceV[f2])  &  (V_new > surfaceV[e2])] = func_fit(V_new[(V_new <= surfaceV[f2])  &  (V_new > surfaceV[e2])],*SVf6)
    fld_h [(V_new <= surfaceV[g2])  &  (V_new > surfaceV[f2])] = func_fit(V_new[(V_new <= surfaceV[g2])  &  (V_new > surfaceV[f2])],*SVf7)
    fld_h [(V_new <= surfaceV[h2])  &  (V_new > surfaceV[f2])] = func_fit(V_new[(V_new <= surfaceV[h2])  &  (V_new > surfaceV[f2])],*SVf8)
    fld_h [(V_new <= surfaceV[i2])  &  (V_new > surfaceV[h2])] = func_fit(V_new[(V_new <= surfaceV[i2])  &  (V_new > surfaceV[h2])],*SVf9)
    fld_h [(V_new <= surfaceV[j2])  &  (V_new > surfaceV[i2])] = func_fit(V_new[(V_new <= surfaceV[j2])  &  (V_new > surfaceV[i2])],*SVf10)
    fld_h [(V_new <= surfaceV[k2])  &  (V_new > surfaceV[j2])] = func_fit(V_new[(V_new <= surfaceV[k2])  &  (V_new > surfaceV[j2])],*SVf11)
    fld_h [(V_new <= surfaceV[l2])  &  (V_new > surfaceV[k2])] = func_fit(V_new[(V_new <= surfaceV[l2])  &  (V_new > surfaceV[k2])],*SVf12)
    fld_h [(V_new <= surfaceV[m2])  &  (V_new > surfaceV[l2])] = func_fit(V_new[(V_new <= surfaceV[m2])  &  (V_new > surfaceV[l2])],*SVf13)
    fld_h [(V_new <= surfaceV[n2])  &  (V_new > surfaceV[m2])] = func_fit(V_new[(V_new <= surfaceV[n2])  &  (V_new > surfaceV[m2])],*SVf14)
    fld_h [(V_new <= surfaceV[o2])  &  (V_new > surfaceV[n2])] = func_fit(V_new[(V_new <= surfaceV[o2])  &  (V_new > surfaceV[n2])],*SVf15)
    fld_h [(V_new <= surfaceV[p2])  &  (V_new > surfaceV[o2])] = func_fit(V_new[(V_new <= surfaceV[p2])  &  (V_new > surfaceV[o2])],*SVf16)
    fld_h [(V_new <= surfaceV[q2])  &  (V_new > surfaceV[p2])] = func_fit(V_new[(V_new <= surfaceV[q2])  &  (V_new > surfaceV[p2])],*SVf17)
    fld_h [(V_new <= surfaceV[r2])  &  (V_new > surfaceV[q2])] = func_fit(V_new[(V_new <= surfaceV[r2])  &  (V_new > surfaceV[q2])],*SVf18)
    fld_h [(V_new <= surfaceV[s2])  &  (V_new > surfaceV[r2])] = func_fit(V_new[(V_new <= surfaceV[s2])  &  (V_new > surfaceV[r2])],*SVf19)
    fld_h [V_new >= surfaceV[s2]] = func_fit(V_new[V_new > surfaceV[s2]],*SVf20)
    fld_h [fld_h < 0]= 0
             
    return fld_h,V_new

def FloodTravel(ssh,V_new,SVf1,SVf2,SVf3,SVf4,SVf5,SVf6,SVf7,SVf8,SVf9,SVf10,SVf11,SVf12,SVf13,SVf14,SVf15,SVf16,SVf17,SVf18,SVf19,SVf20):
    a1 = 0;  a2 = 2;  b1 = 1;  b2 = 3;     c1 = 2;  c2 = 4;  d1 = 3;  d2 = 5
    e1 = 4;  e2 = 6;  f1 = 5;  f2 = 7;     g1 = 6;  g2 = 8;  h1 = 7;  h2 = 9
    i1 = 8;  i2 = 10; j1 = 9;  j2 = 11;    k1 = 10; k2 = 12; l1 = 11; l2 = 13
    m1 = 12; m2 = 14; n1 = 13; n2 = 15;    o1 = 14; o2 = 16; p1 = 15; p2 = 17
    q1 = 16; q2 = 18; r1 = 17; r2 = 19;    s1 = 18; s2 = 20; t1 = 19; t2 = 21
    
    fld_h = np.zeros(np.shape(V_new))
    fld_h [V_new == 0]= 0
    fld_h [(V_new <= surfaceV[a2])  &  (V_new > 0)] = func_fit(V_new[(V_new <= surfaceV[a2])  &  (V_new > 0)],*SVf1)
    fld_h [(V_new <= surfaceV[b2])  &  (V_new > surfaceV[a2])] = func_fit(V_new[(V_new <= surfaceV[b2])  &  (V_new > surfaceV[a2])],*SVf2)
    fld_h [(V_new <= surfaceV[c2])  &  (V_new > surfaceV[b2])] = func_fit(V_new[(V_new <= surfaceV[c2])  &  (V_new > surfaceV[b2])],*SVf3)
    fld_h [(V_new <= surfaceV[d2])  &  (V_new > surfaceV[c2])] = func_fit(V_new[(V_new <= surfaceV[d2])  &  (V_new > surfaceV[c2])],*SVf4)
    fld_h [(V_new <= surfaceV[e2])  &  (V_new > surfaceV[d2])] = func_fit(V_new[(V_new <= surfaceV[e2])  &  (V_new > surfaceV[d2])],*SVf5)
    fld_h [(V_new <= surfaceV[f2])  &  (V_new > surfaceV[e2])] = func_fit(V_new[(V_new <= surfaceV[f2])  &  (V_new > surfaceV[e2])],*SVf6)
    fld_h [(V_new <= surfaceV[g2])  &  (V_new > surfaceV[f2])] = func_fit(V_new[(V_new <= surfaceV[g2])  &  (V_new > surfaceV[f2])],*SVf7)
    fld_h [(V_new <= surfaceV[h2])  &  (V_new > surfaceV[f2])] = func_fit(V_new[(V_new <= surfaceV[h2])  &  (V_new > surfaceV[f2])],*SVf8)
    fld_h [(V_new <= surfaceV[i2])  &  (V_new > surfaceV[h2])] = func_fit(V_new[(V_new <= surfaceV[i2])  &  (V_new > surfaceV[h2])],*SVf9)
    fld_h [(V_new <= surfaceV[j2])  &  (V_new > surfaceV[i2])] = func_fit(V_new[(V_new <= surfaceV[j2])  &  (V_new > surfaceV[i2])],*SVf10)
    fld_h [(V_new <= surfaceV[k2])  &  (V_new > surfaceV[j2])] = func_fit(V_new[(V_new <= surfaceV[k2])  &  (V_new > surfaceV[j2])],*SVf11)
    fld_h [(V_new <= surfaceV[l2])  &  (V_new > surfaceV[k2])] = func_fit(V_new[(V_new <= surfaceV[l2])  &  (V_new > surfaceV[k2])],*SVf12)
    fld_h [(V_new <= surfaceV[m2])  &  (V_new > surfaceV[l2])] = func_fit(V_new[(V_new <= surfaceV[m2])  &  (V_new > surfaceV[l2])],*SVf13)
    fld_h [(V_new <= surfaceV[n2])  &  (V_new > surfaceV[m2])] = func_fit(V_new[(V_new <= surfaceV[n2])  &  (V_new > surfaceV[m2])],*SVf14)
    fld_h [(V_new <= surfaceV[o2])  &  (V_new > surfaceV[n2])] = func_fit(V_new[(V_new <= surfaceV[o2])  &  (V_new > surfaceV[n2])],*SVf15)
    fld_h [(V_new <= surfaceV[p2])  &  (V_new > surfaceV[o2])] = func_fit(V_new[(V_new <= surfaceV[p2])  &  (V_new > surfaceV[o2])],*SVf16)
    fld_h [(V_new <= surfaceV[q2])  &  (V_new > surfaceV[p2])] = func_fit(V_new[(V_new <= surfaceV[q2])  &  (V_new > surfaceV[p2])],*SVf17)
    fld_h [(V_new <= surfaceV[r2])  &  (V_new > surfaceV[q2])] = func_fit(V_new[(V_new <= surfaceV[r2])  &  (V_new > surfaceV[q2])],*SVf18)
    fld_h [(V_new <= surfaceV[s2])  &  (V_new > surfaceV[r2])] = func_fit(V_new[(V_new <= surfaceV[s2])  &  (V_new > surfaceV[r2])],*SVf19)
    fld_h [V_new >= surfaceV[s2]] = func_fit(V_new[V_new > surfaceV[s2]],*SVf20)
    fld_h [fld_h < 0]= 0
    if fld_h > ssh:
        fld_h = ssh
    
    return fld_h

def FloodTravelSectGroup(ssh,sect0,sect1,sect2,sect3,sect_3,sect_2,sect_1,files,V,
                         SVfg1,SVfg2,SVfg3,SVfg4,SVfg5,SVfg6,SVfg7,SVfg8,SVfg9,SVfg10,SVfg11,SVfg12,SVfg13,SVfg14,SVfg15,SVfg16,SVfg17,SVfg18,SVfg19,SVfg20):
    """
    This function propagates the flooding to different divisions within a group
    """
    V_sect = np.zeros(ndiv18)
    fld_h_sect = np.zeros(ndiv18)
    V_sect_avr = np.zeros(ndiv18)
 
    # ID3~14
    for i in range(12):
        m = int(sect3[i][3])
        for s in sect3[i]:
            s = int(s)
            V_sect[m] = V_sect[m] + V[s]
        V_sect_avr[m] = V_sect[m]/7

    #ID0
    i = 10; 
    for s in sect0:
        s = int(s)
        V_sect[i] = V_sect[i] + V[s]
    V_sect_avr[i] = V_sect[i]/4

    #ID1
    i = 11; 
    for s in sect1:
        s = int(s)
        V_sect[i] = V_sect[i] + V[s]
    V_sect_avr[i] = V_sect[i]/5

    #ID2
    i = 5;
    for s in sect2:
        s = int(s)
        V_sect[i] = V_sect[i] + V[s]
    V_sect_avr[i] = V_sect[i]/6

    #ID15
    i = 7;
    for s in sect_3:
        s = int(s)
        V_sect[i] = V_sect[i] + V[s]
    V_sect_avr[i] = V_sect[i]/6

    #ID16
    i = 2;
    for s in sect_2:
        s = int(s)
        V_sect[i] = V_sect[i] + V[s]
    V_sect_avr[i] = V_sect[i]/5

    #ID17
    i = 6;
    for s in sect_1:
        s = int(s)
        V_sect[i] = V_sect[i] + V[s]
    V_sect_avr[i] = V_sect[i]/4

    for i in range(ndiv18):    
        # redistribute the water
        fld_h_sect[i] = FloodTravel(ssh,V_sect[i],SVfg1[i,:],SVfg2[i,:],SVfg3[i,:],SVfg4[i,:],SVfg5[i,:],SVfg6[i,:],SVfg7[i,:],SVfg8[i,:],SVfg9[i,:],SVfg10[i,:],SVfg11[i,:],SVfg12[i,:],SVfg13[i,:],SVfg14[i,:],SVfg15[i,:],SVfg16[i,:],SVfg17[i,:],SVfg18[i,:],SVfg19[i,:],SVfg20[i,:])

    fld_h_sect[fld_h_sect<0] = 0
    
    return fld_h_sect,V_sect_avr



## Import Data

In [4]:
# Topography Data
topo   = pd.read_csv(r"Data\LMN_div.csv")
elev   = topo["MEAN"]
fid    = topo["FID"]
div18  = topo["DIV18"]
ndiv18 = np.unique(div18).size

roughness = pd.read_csv(r"Data\LMN_Roughness.csv")["Roughness"]
slope     = pd.read_csv(r"Data\LMN_Slope.csv")["Slope"]

# Import Surface Volume
files = glob.glob(r'Data\NewSurfaceVolumeCombined\LMN_div18_new_*.csv') 
groupcsvfiles = glob.glob(r'Data\NewSurfaceVolumeGrouped\LMN_div18_new_grouped_*.csv') 

H = np.append(np.linspace(0,3,13),np.linspace(3.5,7,8))

l        = 100  # segment unit length
sections = [10,11,5,12,1,13,16,4,17,15,3,14,0,9,8,7,2,6] #div number, in sequential geographic order from east: north to south, then west: north to south

# Setup - Division Connectivities
# these sections give the groupings of different divisions when propagating flood water
sect3 = np.zeros([len(sections)-6,7])
k = 3
for i in sections[3:-3]:
    sect3[k-3] = [sections[k-3],sections[k-2],sections[k-1],i,sections[k+1],sections[k+2],sections[k+3]]
    k = k + 1

sect0  = [sections[0],sections[1],sections[2],sections[3]] 
sect1  = [sections[0],sections[1],sections[2],sections[3],sections[4]]
sect2  = [sections[0],sections[1],sections[2],sections[3],sections[4],sections[5]]
sect_3 = [sections[-6],sections[-5],sections[-4],sections[-3],sections[-2],sections[-1]]
sect_2 = [sections[-5],sections[-4],sections[-3],sections[-2],sections[-1]]
sect_1 = [sections[-4],sections[-3],sections[-2],sections[-1]]

#Surface Volume functions for grouped divisions
SVfg1 = np.zeros([ndiv18,2]);  SVfg2 = np.zeros([ndiv18,2]);  SVfg3 = np.zeros([ndiv18,2]);  SVfg4 = np.zeros([ndiv18,2])
SVfg5 = np.zeros([ndiv18,2]);  SVfg6 = np.zeros([ndiv18,2]);  SVfg7 = np.zeros([ndiv18,2]);  SVfg8 = np.zeros([ndiv18,2])
SVfg9 = np.zeros([ndiv18,2]);  SVfg10 = np.zeros([ndiv18,2]); SVfg11 = np.zeros([ndiv18,2]); SVfg12 = np.zeros([ndiv18,2])
SVfg13 = np.zeros([ndiv18,2]); SVfg14 = np.zeros([ndiv18,2]); SVfg15 = np.zeros([ndiv18,2]); SVfg16 = np.zeros([ndiv18,2])
SVfg17 = np.zeros([ndiv18,2]); SVfg18 = np.zeros([ndiv18,2]); SVfg19 = np.zeros([ndiv18,2]); SVfg20= np.zeros([ndiv18,2])

i = 0

for f in groupcsvfiles:
    surfaceVg = pd.read_csv(f)["volume"]
    SVfg1[i,:],SVfg2[i,:],SVfg3[i,:],SVfg4[i,:],SVfg5[i,:],SVfg6[i,:],SVfg7[i,:],SVfg8[i,:],SVfg9[i,:],SVfg10[i,:],SVfg11[i,:],SVfg12[i,:],SVfg13[i,:],SVfg14[i,:],SVfg15[i,:],SVfg16[i,:],SVfg17[i,:],SVfg18[i,:],SVfg19[i,:],SVfg20[i,:] = SurfaceVolFunc(surfaceVg,H)
    i = i + 1 

#Surface Volume functions for individual divisions
SVf1 = np.zeros([ndiv18,2]);  SVf2 = np.zeros([ndiv18,2]);  SVf3 = np.zeros([ndiv18,2]);  SVf4 = np.zeros([ndiv18,2])
SVf5 = np.zeros([ndiv18,2]);  SVf6 = np.zeros([ndiv18,2]);  SVf7 = np.zeros([ndiv18,2]);  SVf8 = np.zeros([ndiv18,2])
SVf9 = np.zeros([ndiv18,2]);  SVf10 = np.zeros([ndiv18,2]); SVf11 = np.zeros([ndiv18,2]); SVf12 = np.zeros([ndiv18,2])
SVf13 = np.zeros([ndiv18,2]); SVf14 = np.zeros([ndiv18,2]); SVf15 = np.zeros([ndiv18,2]); SVf16 = np.zeros([ndiv18,2])
SVf17 = np.zeros([ndiv18,2]); SVf18 = np.zeros([ndiv18,2]); SVf19 = np.zeros([ndiv18,2]); SVf20= np.zeros([ndiv18,2])

SV_all = []

i = 0
for f in files:
    surfaceV_height = pd.read_csv(f)
    surfaceV = surfaceV_height["volume"]
    SV_all = np.append(SV_all,surfaceV)
    SVf1[i,:],SVf2[i,:],SVf3[i,:],SVf4[i,:],SVf5[i,:],SVf6[i,:],SVf7[i,:],SVf8[i,:],SVf9[i,:],SVf10[i,:],SVf11[i,:],SVf12[i,:],SVf13[i,:],SVf14[i,:],SVf15[i,:],SVf16[i,:],SVf17[i,:],SVf18[i,:],SVf19[i,:],SVf20[i,:] = SurfaceVolFunc(surfaceV,H)
    i = i+1
    
SV_all = SV_all.reshape(18,21)


C:\Users\Mimi\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


## Flood Simulation - No Protective Measures

Flood simulation takes in storm peaks, surge heights, and time, as well as information about the terrain, and the surface volume functions from the Import Data cell to obtain the floodheights of individual cells and redistribute the flooding to adjacent cells. The resulting flood heights of the storm for cold and warm storms are outputted in a csv.  

In [5]:
s_i = 0 # hurricane data 

start_time = time.time()

# Storm Surge imports
peak_c  = pd.read_csv(r"Data\SurgeData\peak_c.csv").values[s_i]
peak_w  = pd.read_csv(r"Data\SurgeData\peak_w.csv").values[s_i]
surge_c = pd.read_csv(r"Data\SurgeData\surge_c.csv").values[s_i]
surge_w = pd.read_csv(r"Data\SurgeData\surge_w.csv").values[s_i]
time_c  = pd.read_csv(r"Data\SurgeData\time_c.csv").values[s_i]
time_w  = pd.read_csv(r"Data\SurgeData\time_w.csv").values[s_i]

time1_c  = time_c[:10]
time2_c  = time_c[10:]
cpi1_c   = surge_c[:10]
cpi2_c   = surge_c[10:]

time1_w  = time_w[:10]
time2_w  = time_w[10:]
cpi1_w   = surge_w[:10]
cpi2_w   = surge_w[10:]

fld_h_c = np.zeros(ndiv18);  fld_h_w = np.zeros(ndiv18)
V_c     = np.zeros(ndiv18);  V_w     = np.zeros(ndiv18)

#Function calls

#Flood height calculation
i = 0
for i in range(18):
    fld_h_w[i],V_w[i] = FloodHeight(SV_all[i],slope[i],roughness[i],SVf1[i,:],SVf2[i,:],SVf3[i,:],SVf4[i,:],SVf5[i,:],SVf6[i,:],SVf7[i,:],SVf8[i,:],SVf9[i,:],SVf10[i,:],SVf11[i,:],SVf12[i,:],SVf13[i,:],SVf14[i,:],SVf15[i,:],SVf16[i,:],SVf17[i,:],SVf18[i,:],SVf19[i,:],SVf20[i,:],time1_w,time2_w,cpi1_w,cpi2_w,nt,elev[div18==i],fid[div18==i],l,peak_w,i)
    fld_h_c[i],V_c[i] = FloodHeight(SV_all[i],slope[i],roughness[i],SVf1[i,:],SVf2[i,:],SVf3[i,:],SVf4[i,:],SVf5[i,:],SVf6[i,:],SVf7[i,:],SVf8[i,:],SVf9[i,:],SVf10[i,:],SVf11[i,:],SVf12[i,:],SVf13[i,:],SVf14[i,:],SVf15[i,:],SVf16[i,:],SVf17[i,:],SVf18[i,:],SVf19[i,:],SVf20[i,:],time1_c,time2_c,cpi1_c,cpi2_c,nt,elev[div18==i],fid[div18==i],l,peak_c,i)

# redistribution - group
fld_h_w_sect_g,V_w_sect_avr = FloodTravelSectGroup(peak_w,sect0,sect1,sect2,sect3,sect_3,sect_2,sect_1,files,V_w,SVfg1,SVfg2,SVfg3,SVfg4,SVfg5,SVfg6,SVfg7,SVfg8,SVfg9,SVfg10,SVfg11,SVfg12,SVfg13,SVfg14,SVfg15,SVfg16,SVfg17,SVfg18,SVfg19,SVfg20)
fld_h_c_sect_g,V_c_sect_avr = FloodTravelSectGroup(peak_c,sect0,sect1,sect2,sect3,sect_3,sect_2,sect_1,files,V_c,SVfg1,SVfg2,SVfg3,SVfg4,SVfg5,SVfg6,SVfg7,SVfg8,SVfg9,SVfg10,SVfg11,SVfg12,SVfg13,SVfg14,SVfg15,SVfg16,SVfg17,SVfg18,SVfg19,SVfg20)

with open('flood_heights_w.csv','w') as out:
    write = csv.writer(out)
    write.writerow(fld_h_w_sect_g)
with open('flood_heights_c.csv','w') as out:
    write = csv.writer(out)
    write.writerow(fld_h_c_sect_g)

elapsed_time = time.time() - start_time
print("Elapsed Time: ",elapsed_time, "sec")            
print("Complete!")

Elapsed Time:  0.5505294799804688 sec
Complete!


## Flood Simulation - BigU Lower East Side

This cell takes in storm parameters and surface volumes as well as parameters for a sea wall called the BigU and calculates the flood heights given with the effects of the BigU. The resulting output are csv files for warm and cold storm flood heights.

In [6]:
s_i = 0 # hurricane data 

start_time = time.time()

topo = pd.read_csv(r"Data\LMN_div.csv")
elev = topo["MEAN"]
BigU_les = pd.read_csv(r"Data\BigU_LES_all.csv")["ID"] #this csv specifies the locations of the wall
wallh_bigU = 15 * ftm #the "15" specifies the height of the wall at 15 ft
elev[BigU_les] = elev[BigU_les] + wallh_bigU

# Storm Surge imports
peak_c  = pd.read_csv(r"Data\SurgeData\peak_c.csv").values[s_i]
peak_w  = pd.read_csv(r"Data\SurgeData\peak_w.csv").values[s_i]
surge_c = pd.read_csv(r"Data\SurgeData\surge_c.csv").values[s_i]
surge_w = pd.read_csv(r"Data\SurgeData\surge_w.csv").values[s_i]
time_c  = pd.read_csv(r"Data\SurgeData\time_c.csv").values[s_i]
time_w  = pd.read_csv(r"Data\SurgeData\time_w.csv").values[s_i]

time1_c  = time_c[:10]
time2_c  = time_c[10:]
cpi1_c   = surge_c[:10]
cpi2_c   = surge_c[10:]

time1_w  = time_w[:10]
time2_w  = time_w[10:]
cpi1_w   = surge_w[:10]
cpi2_w   = surge_w[10:]

fld_h_c = np.zeros(ndiv18);  fld_h_w = np.zeros(ndiv18)
V_c     = np.zeros(ndiv18);  V_w     = np.zeros(ndiv18)

#Function Calls

#Flood height calculation with walls
i = 0
for i in range(18):
    fld_h_w[i],V_w[i] = FloodHeightWall(SV_all[i],BigU_les,slope[i],roughness[i],SVf1[i,:],SVf2[i,:],SVf3[i,:],SVf4[i,:],SVf5[i,:],SVf6[i,:],SVf7[i,:],SVf8[i,:],SVf9[i,:],SVf10[i,:],SVf11[i,:],SVf12[i,:],SVf13[i,:],SVf14[i,:],SVf15[i,:],SVf16[i,:],SVf17[i,:],SVf18[i,:],SVf19[i,:],SVf20[i,:],time1_w,time2_w,cpi1_w,cpi2_w,nt,elev[div18==i],fid[div18==i],l,peak_w,i)
    fld_h_c[i],V_c[i] = FloodHeightWall(SV_all[i],BigU_les,slope[i],roughness[i],SVf1[i,:],SVf2[i,:],SVf3[i,:],SVf4[i,:],SVf5[i,:],SVf6[i,:],SVf7[i,:],SVf8[i,:],SVf9[i,:],SVf10[i,:],SVf11[i,:],SVf12[i,:],SVf13[i,:],SVf14[i,:],SVf15[i,:],SVf16[i,:],SVf17[i,:],SVf18[i,:],SVf19[i,:],SVf20[i,:],time1_c,time2_c,cpi1_c,cpi2_c,nt,elev[div18==i],fid[div18==i],l,peak_c,i)

# redistribution - group
fld_h_w_sect_g,V_w_sect_avr = FloodTravelSectGroup(peak_w,sect0,sect1,sect2,sect3,sect_3,sect_2,sect_1,files,V_w,SVfg1,SVfg2,SVfg3,SVfg4,SVfg5,SVfg6,SVfg7,SVfg8,SVfg9,SVfg10,SVfg11,SVfg12,SVfg13,SVfg14,SVfg15,SVfg16,SVfg17,SVfg18,SVfg19,SVfg20)
fld_h_c_sect_g,V_c_sect_avr = FloodTravelSectGroup(peak_c,sect0,sect1,sect2,sect3,sect_3,sect_2,sect_1,files,V_c,SVfg1,SVfg2,SVfg3,SVfg4,SVfg5,SVfg6,SVfg7,SVfg8,SVfg9,SVfg10,SVfg11,SVfg12,SVfg13,SVfg14,SVfg15,SVfg16,SVfg17,SVfg18,SVfg19,SVfg20)

with open('flood_heights_bigU_w.csv','w') as out:
    write = csv.writer(out)
    write.writerow(fld_h_w_sect_g)
with open('flood_heights_bigU_c.csv','w') as out:
    write = csv.writer(out)
    write.writerow(fld_h_c_sect_g)

elapsed_time = time.time() - start_time
print("Elapsed Time: ",elapsed_time, "sec")            
print("Complete!")




Elapsed Time:  0.42421770095825195 sec
Complete!


<ipython-input-6-7e8a80684bc3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elev[BigU_les] = elev[BigU_les] + wallh_bigU
